<a href="https://www.kaggle.com/code/kiranpatil7022/employee-salary-insights-and-predictions?scriptVersionId=148198082" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## **1. Loading libraries and data**

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ai-robotics-employee-salary/train_dataset.csv
/kaggle/input/ai-robotics-employee-salary/test_dataset.csv
/kaggle/input/ai-robotics-employee-salary/train_salaries.csv


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [3]:
# load the train_dataset, train_salaries, test_dataset
train_data = pd.read_csv("/kaggle/input/ai-robotics-employee-salary/train_dataset.csv")
train_data_salaries = pd.read_csv("/kaggle/input/ai-robotics-employee-salary/train_salaries.csv")
#test_data
df_test = pd.read_csv("/kaggle/input/ai-robotics-employee-salary/test_dataset.csv")

In [4]:
train_data.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16


In [5]:
train_data_salaries.head()

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163


In [6]:
df_test.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
1,JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
2,JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9
3,JOB1362685407690,COMP21,MANAGER,HIGH_SCHOOL,NONE,OIL,14,96
4,JOB1362685407691,COMP36,JUNIOR,DOCTORAL,BIOLOGY,OIL,10,44


In [7]:
# merging train_data and train_data_salaries datasets
df = pd.merge(train_data,train_data_salaries,on="jobId")
df

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163
...,...,...,...,...,...,...,...,...,...
999995,JOB1362685407682,COMP56,VICE_PRESIDENT,BACHELORS,CHEMISTRY,HEALTH,19,94,88
999996,JOB1362685407683,COMP24,CTO,HIGH_SCHOOL,NONE,FINANCE,12,35,160
999997,JOB1362685407684,COMP23,JUNIOR,HIGH_SCHOOL,NONE,EDUCATION,16,81,64
999998,JOB1362685407685,COMP3,CFO,MASTERS,NONE,HEALTH,6,5,149


## **2. Exploratory Data Analysis**

In [8]:
df.info()
# no non-null values present

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
 8   salary               1000000 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 68.7+ MB


In [9]:
df.describe()

,yearsExperience,milesFromMetropolis,salary
count,1000000.000000,1000000.000000,1000000.000000
mean,11.992386,49.529260,116.061818
std,7.212391,28.877733,38.717936
min,0.000000,0.000000,0.000000
25%,6.000000,25.000000,88.000000
50%,12.000000,50.000000,114.000000
75%,18.000000,75.000000,141.000000
max,24.000000,99.000000,301.000000


In [10]:
df[df["salary"]==0]

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
30559,JOB1362684438246,COMP44,JUNIOR,DOCTORAL,MATH,AUTO,11,7,0
495984,JOB1362684903671,COMP34,JUNIOR,NONE,NONE,OIL,1,25,0
652076,JOB1362685059763,COMP25,CTO,HIGH_SCHOOL,NONE,AUTO,6,60,0
816129,JOB1362685223816,COMP42,MANAGER,DOCTORAL,ENGINEERING,FINANCE,18,6,0
828156,JOB1362685235843,COMP40,VICE_PRESIDENT,MASTERS,ENGINEERING,WEB,3,29,0


In [11]:
df[df["salary"]<50]

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
8,JOB1362684407695,COMP20,JANITOR,HIGH_SCHOOL,NONE,EDUCATION,1,54,31
14,JOB1362684407701,COMP57,JANITOR,NONE,NONE,AUTO,21,81,47
16,JOB1362684407703,COMP14,JUNIOR,MASTERS,PHYSICS,SERVICE,1,91,47
20,JOB1362684407707,COMP44,JANITOR,NONE,NONE,SERVICE,11,96,32
119,JOB1362684407806,COMP15,JANITOR,HIGH_SCHOOL,NONE,WEB,6,60,49
...,...,...,...,...,...,...,...,...,...
999860,JOB1362685407547,COMP61,JANITOR,NONE,NONE,EDUCATION,2,37,45
999881,JOB1362685407568,COMP16,JANITOR,HIGH_SCHOOL,NONE,HEALTH,2,61,46
999898,JOB1362685407585,COMP13,JANITOR,HIGH_SCHOOL,NONE,WEB,7,99,41
999901,JOB1362685407588,COMP27,JANITOR,HIGH_SCHOOL,NONE,SERVICE,4,46,35


In [12]:
df[df["jobType"]=="JANITOR"].describe()

,yearsExperience,milesFromMetropolis,salary
count,124971.000000,124971.000000,124971.000000
mean,11.969657,49.443815,70.813045
std,7.211544,28.842493,24.643021
min,0.000000,0.000000,17.000000
25%,6.000000,24.000000,52.000000
50%,12.000000,50.000000,68.000000
75%,18.000000,74.000000,86.000000
max,24.000000,99.000000,189.000000


In [13]:
df[df['salary']<31]

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
461,JOB1362684408148,COMP14,JANITOR,HIGH_SCHOOL,NONE,EDUCATION,1,73,23
873,JOB1362684408560,COMP35,JANITOR,NONE,NONE,AUTO,1,84,29
1195,JOB1362684408882,COMP1,JANITOR,HIGH_SCHOOL,NONE,EDUCATION,2,59,28
1397,JOB1362684409084,COMP53,JANITOR,NONE,NONE,EDUCATION,6,90,28
2003,JOB1362684409690,COMP9,JANITOR,NONE,NONE,EDUCATION,1,82,24
...,...,...,...,...,...,...,...,...,...
998124,JOB1362685405811,COMP31,JANITOR,HIGH_SCHOOL,NONE,EDUCATION,1,74,25
999502,JOB1362685407189,COMP29,JANITOR,NONE,NONE,EDUCATION,2,67,30
999706,JOB1362685407393,COMP28,JANITOR,NONE,NONE,SERVICE,1,86,28
999754,JOB1362685407441,COMP23,JANITOR,HIGH_SCHOOL,NONE,SERVICE,4,81,26


In [14]:
# salaries less then 30 can be removed considering for my target having done a degree its the minimum salary.
df = df[df['salary']>30]
df

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83,130
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73,101
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38,137
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17,142
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16,163
...,...,...,...,...,...,...,...,...,...
999995,JOB1362685407682,COMP56,VICE_PRESIDENT,BACHELORS,CHEMISTRY,HEALTH,19,94,88
999996,JOB1362685407683,COMP24,CTO,HIGH_SCHOOL,NONE,FINANCE,12,35,160
999997,JOB1362685407684,COMP23,JUNIOR,HIGH_SCHOOL,NONE,EDUCATION,16,81,64
999998,JOB1362685407685,COMP3,CFO,MASTERS,NONE,HEALTH,6,5,149


In [15]:
df.columns

Index(['jobId', 'companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis', 'salary'],
      dtype='object')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 997548 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   jobId                997548 non-null  object
 1   companyId            997548 non-null  object
 2   jobType              997548 non-null  object
 3   degree               997548 non-null  object
 4   major                997548 non-null  object
 5   industry             997548 non-null  object
 6   yearsExperience      997548 non-null  int64 
 7   milesFromMetropolis  997548 non-null  int64 
 8   salary               997548 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 76.1+ MB


In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   jobId                1000000 non-null  object
 1   companyId            1000000 non-null  object
 2   jobType              1000000 non-null  object
 3   degree               1000000 non-null  object
 4   major                1000000 non-null  object
 5   industry             1000000 non-null  object
 6   yearsExperience      1000000 non-null  int64 
 7   milesFromMetropolis  1000000 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 61.0+ MB


In [18]:
df.describe()

,yearsExperience,milesFromMetropolis,salary
count,997548.000000,997548.000000,997548.000000
mean,12.015214,49.458372,116.280462
std,7.204992,28.863518,38.512936
min,0.000000,0.000000,31.000000
25%,6.000000,24.000000,89.000000
50%,12.000000,49.000000,114.000000
75%,18.000000,74.000000,141.000000
max,24.000000,99.000000,301.000000


In [19]:
df_test.describe()

,yearsExperience,milesFromMetropolis
count,1000000.000000,1000000.000000
mean,12.002104,49.526414
std,7.213179,28.889713
min,0.000000,0.000000
25%,6.000000,25.000000
50%,12.000000,50.000000
75%,18.000000,75.000000
max,24.000000,99.000000


In [20]:
print("train data length: ",len(df))
print("test data length: ",len(df_test))

train data length:  997548
test data length:  1000000


In [21]:
# get unique values in train data set
print("unique values in train dataset\n")
for x in df.columns:
    print(x,":",len(df[x].unique()))

unique values in train dataset

jobId : 997548
companyId : 63
jobType : 8
degree : 5
major : 9
industry : 7
yearsExperience : 25
milesFromMetropolis : 100
salary : 265


In [22]:
# get unique values in test data set
print("unique values in test dataset\n")
for x in df_test.columns:
    print(x,":",len(df_test[x].unique()))

unique values in test dataset

jobId : 1000000
companyId : 63
jobType : 8
degree : 5
major : 9
industry : 7
yearsExperience : 25
milesFromMetropolis : 100
